# Classical feedforward and control flow

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.0.0
```
</details>

Dynamic circuits are powerful tools that let us measure qubits in the middle of a quantum circuit execution and then perform classical logical operations within the circuit, based on the outcome of those mid-circuit measurements.  This process is also known as _classical feedforward_. While we are still in the early days of understanding how best to take advantage of dynamic circuits, the quantum research community has already identified a number of use cases, such as the following:

* Efficient quantum state preparation, such as [W-state](https://arxiv.org/pdf/2412.07969) and [MPS state](https://arxiv.org/abs/2404.16083)
* [Efficient long-range entanglement](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.5.030339) between qubits on the same chip by using shallow circuits
* Efficient [sampling of IQP-like circuits](https://arxiv.org/pdf/2505.04705)

These improvements brought by dynamic circuits, however, come with trade-offs. Mid-circuit measurements and classical operations typically have longer execution time than two-qubit gates, and this increase in time might negate the benefits of reduced circuit depth. Therefore, reducing the length of mid-circuit measurement is a focus area of improvement as IBM Quantum rolls out the [new version](/announcements/product-updates/2025-03-03-new-version-dynamic-circuits) of dynamic circuits.

The [OpenQASM3 specification](https://openqasm.com/language/classical.html#looping-and-branching) defines a number of conditional instructions, but Qiskit Runtime currently only supports the [if_test](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#if_test) statement, which can be used as a method on [QuantumCircuit.](/docs/api/qiskit/qiskit.circuit.QuantumCircuit) This methods returns a [context manager](https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers) and is typically used in a `with` statement. This guide describes how to use this conditional statement.

<Admonition type="caution">
  - There are some limitations of classical feedforward and control flow operations on quantum hardware that might impact your program. For more information, see [Hardware considerations and limitations for classical feedforward and control flow](./dynamic-circuits-considerations).
  - See the [QASM3 feature table](/docs/guides/qasm-feature-table) to determine which features are supported on Qiskit and Qiskit Runtime.
</Admonition>

## If statement

The if statement is used to conditionally perform operations based on the value of a classical bit or register.

In the example below, we apply a Hadamard gate to a qubit and measure it. If the result is 1, then we apply an X gate on the qubit, which has the effect of flipping it back to the 0 state. We then measure the qubit again. The resulting measurement outcome should be 0 with 100% probability.

In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister

qubits = QuantumRegister(1)
clbits = ClassicalRegister(1)
circuit = QuantumCircuit(qubits, clbits)
(q0,) = qubits
(c0,) = clbits

circuit.h(q0)
circuit.measure(q0, c0)
with circuit.if_test((c0, 1)):
    circuit.x(q0)
circuit.measure(q0, c0)
circuit.draw("mpl")

# example output counts: {'0': 1024}

<Image src="/docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/008f85d9-5b48-4e84-b250-846b07901607-0.svg" alt="Output of the previous code cell" />

The `with` statement can be given an assignment target which is itself a context manager that can be stored and subsequently used to create an else block, which is executed whenever the contents of the if block are *not* executed.

In the example below, we initialize registers with two qubits and two classical bits. We apply a Hadamard gate to the first qubit and measure it. If the result is 1, then we apply a Hadamard gate on the second qubit; otherwise, we apply an X gate on the second qubit. Finally, we measure the second qubit as well.

In [2]:
qubits = QuantumRegister(2)
clbits = ClassicalRegister(2)
circuit = QuantumCircuit(qubits, clbits)
(q0, q1) = qubits
(c0, c1) = clbits

circuit.h(q0)
circuit.measure(q0, c0)
with circuit.if_test((c0, 1)) as else_:
    circuit.h(q1)
with else_:
    circuit.x(q1)
circuit.measure(q1, c1)

circuit.draw("mpl")

# example output counts: {'01': 260, '11': 272, '10': 492}

<Image src="/docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/5b22ae93-27d1-4a0c-89e2-9f88383039d6-0.svg" alt="Output of the previous code cell" />

In addition to conditioning on a single classical bit, it's also possible to condition on the value of a classical register composed of multiple bits.

In the example below, we apply Hadamard gates to two qubits and measure them. If the result is `01`, that is, the first qubit is 1 and the second qubit is 0, then we apply an X gate to a third qubit. Finally, we measure the third qubit. Note that for clarity, we chose to specify the state of the third classical bit, which is 0, in the if condition. In the circuit drawing, the condition is indicated by the circles on the classical bits being conditioned on. A black circle indicates conditioning on 1, while a white circle indicates conditioning on 0.

In [3]:
qubits = QuantumRegister(3)
clbits = ClassicalRegister(3)
circuit = QuantumCircuit(qubits, clbits)
(q0, q1, q2) = qubits
(c0, c1, c2) = clbits

circuit.h([q0, q1])
circuit.measure(q0, c0)
circuit.measure(q1, c1)
with circuit.if_test((clbits, 0b001)):
    circuit.x(q2)
circuit.measure(q2, c2)

circuit.draw("mpl")

# example output counts: {'101': 269, '011': 260, '000': 252, '010': 243}

<Image src="/docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/15ac9465-c61f-4de1-81e8-e4edfafb07b4-0.svg" alt="Output of the previous code cell" />

## Classical expressions

The Qiskit classical expression module [`qiskit.circuit.classical`](/docs/api/qiskit/circuit_classical) contains an exploratory representation of runtime operations on classical values during circuit execution. Currently, only simple expressions on bits and registers that result in a Boolean value are supported, and these are only valid for use in the condition of `QuantumCircuit.if_test()` (`IfElseOp`).

The following example shows you can use the calculation of the parity to create an n-qubit GHZ state using dynamic circuits. First, generate $n/2$ Bell pairs on
adjacent qubits. Then, glue these pairs together using a layer of CNOT gates in between pairs. You then measure the target qubit of all prior CNOT gates and reset each measured qubit to the state $\vert 0 \rangle$. You apply $X$ to every unmeasured site for which the parity of all preceding bits is odd. Finally, CNOT gates are applied to the measured qubits to re-establish the entanglement lost in the measurement.


In the parity calculation, the first element of the constructed expression involves lifting the Python object `mr[0]`to a [`Value`](/docs/api/qiskit/circuit_classical#value)  node. This is not necessary for `mr[1]` and the possible following classical register, as they are inputs to `expr.bit_xor`, and any necessary lifting is done automatically in these cases. Such expressions can be built up in loops and other constructs.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.classical import expr

num_qubits = 8
if num_qubits % 2 or num_qubits < 4:
    raise ValueError("num_qubits must be an even integer ≥ 4")
meas_qubits = list(range(2, num_qubits, 2))  # qubits to measure and reset

qr = QuantumRegister(num_qubits, "qr")
mr = ClassicalRegister(len(meas_qubits), "m")
qc = QuantumCircuit(qr, mr)

# Create local Bell pairs
qc.reset(qr)
qc.h(qr[::2])
for ctrl in range(0, num_qubits, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

# Glue neighboring pairs
for ctrl in range(1, num_qubits - 1, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

# Measure boundary qubits between pairs,reset to 0
for k, q in enumerate(meas_qubits):
    qc.measure(qr[q], mr[k])
    qc.reset(qr[q])

# Parity-conditioned X corrections
# Each non-measured qubit gets flipped iff the parity (XOR) of all
# preceding measurement bits is 1
for tgt in range(num_qubits):
    if tgt in meas_qubits:  # skip measured qubits
        continue
    # all measurement registers whose physical qubit index < tgt
    left_bits = [k for k, q in enumerate(meas_qubits) if q < tgt]
    if not left_bits:  # skip if list empty
        continue

    # build XOR-parity expression
    parity = expr.lift(mr[left_bits[0]])  # lift the first bit to Value
    for k in left_bits[1:]:
        parity = expr.bit_xor(
            mr[k], parity
        )  # calculate parity with all other bits
    with qc.if_test(parity):  # Add X if parity is 1
        qc.x(qr[tgt])

# Re-entangle measured qubits
for ctrl in range(1, num_qubits - 1, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

In [23]:
qc.draw(output="mpl", style="iqp", idle_wires=False, fold=-1)

<Image src="/docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/3bdb923d-c466-4743-99b1-d46fda9660ef-0.avif" alt="Output of the previous code cell" />

## Limitations

Be aware of the following constraints when using dynamic circuits:

- Whenever you do a measure followed by an `if (c)`, the values that are measured and subsequently used in the if condition has a limit of **60 bits**.
- Nested conditionals are not allowed.
- Having `reset` inside conditionals is not supported.

## Next steps

<Admonition type="tip" title="Recommendations">
-  See an example of dynamic circuits in the [Repeat until success](/docs/tutorials/repeat-until-success) tutorial.
- Learn about [Deferred timing resolution using stretch](/docs/guides/stretch)
- For considerations and limitations related to running dynamic circuits on quantum hardware, see the [Hardware considerations and limitations for classical feedforward and control flow](/docs/guides/dynamic-circuits-considerations) guide.
-  Review the [circuit library API](/docs/api/qiskit/circuit_library) reference.
</Admonition>